<a href="https://colab.research.google.com/github/VIOLDAVE/my_project/blob/main/Massive_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project requirement: https://docs.google.com/document/d/1GkK28wOyjTPZEZuOumKPU0z1NzVT_9sxETjPOcLPVYo/edit?tab=t.0#heading=h.qifoo7co6qtd

Professor : https://malchiodi.di.unimi.it/teaching/AMD-DSE/2024-25/en

In [1]:
import os

if not os.path.exists("my_project"):
    !git clone https://github.com/VIOLDAVE/my_project.git

if os.path.exists("my_project"):
    os.chdir("my_project")
else:
    raise FileNotFoundError("Failed to find or clone the 'my_project' directory.")

Cloning into 'my_project'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 38 (delta 19), reused 18 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 25.87 KiB | 1.18 MiB/s, done.
Resolving deltas: 100% (19/19), done.


Install packages.

In [ ]:
!pip install -r requirement.txt

 kaggle  set up and Authentication

In [2]:
import json, os
from google.colab import files

# Upload the file
uploaded = files.upload()

# Process the uploaded file
for fn in uploaded.keys():
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

    # Read credentials from the uploaded file
    with open(fn, 'r') as f:
        creds = json.load(f)
        os.environ['KAGGLE_USERNAME'] = 'violaawor2'
        os.environ['KAGGLE_KEY'] = 'f93b76093cd1863d38f50e4da7642437'

    # Move and rename the uploaded file to the correct kaggle.json path
    os.makedirs("/root/.kaggle", exist_ok=True)
    os.rename(fn, "/root/.kaggle/kaggle.json")
    os.chmod("/root/.kaggle/kaggle.json", 0o600)


Saving kaggle_2.json to kaggle_2 (1).json
User uploaded file "kaggle_2 (1).json" with length 66 bytes


dowloading files

In [3]:
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
100% 1.06G/1.06G [00:05<00:00, 201MB/s]
100% 1.06G/1.06G [00:05<00:00, 192MB/s]


unzipping the dataset

In [20]:
import zipfile
import os

zip_file_path = "/content/my_project/amazon-books-reviews.zip"
extract_path = "/content/data"

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

# List the extracted files to confirm
!ls /content/data

Files extracted to: /content/data
books_data.csv	Books_rating.csv


Loading the dataset on Spark

In [21]:
# Load the dataset with spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()
books_rating = spark.read.csv("/content/data/Books_rating.csv", header=True, inferSchema=True)

Show a few raws of the dataset

In [22]:
books_rating.show(5)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| NULL|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| NULL|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

creating a sample of dataset becase the file is too heavy for my colab space

In [23]:
sample = 0.02 # can be adjusted as needed

In [24]:
books_sample = books_rating.sample(fraction=sample, seed=42)
books_sample.show(10)
size = books_sample.count()

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|0595344550|Whispers of the W...|10.95| AUR0VA5H0C66C|"LoveToRead ""Act...|               1/2|         1.0| 1119225600|        Buyer beware|"This is a self-p...|
|B000NKGYMK|    Alaska Sourdough| NULL|A258YNWJW2264M|"Tessa F. Briggs ...|              8/14|         3.0| 1241827200|Not your quick re...|"After having a c...|
|B000NKGYMK|    Alaska Sourdough| NULL|A2WY5VMJQ0MM1A|                 Aoy|               0/0|         5.0| 1351641600|        Old and good|This book is wort...|
|157067051X|Voices from the 

In [25]:
size

60498

Checking null valuse,. I will use the review column which has strings.

In [26]:
# Checking  for null values in the 'review/text' column
null_df = books_rating.filter(
    books_rating['review/text'].isNull() | (books_rating['review/text'].rlike('^\s*$'))
)

null_df.show(15)


+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|        Id|               Title|               Price|             User_id|         profileName|  review/helpfulness|        review/score|         review/time|      review/summary|review/text|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|B00085T7O2|"""Catch 'em aliv...|",,ASEH0CVYVGZCZ,...|                 0/1|                 5.0|          1341705600|Fascinating life ...|There are many id...|                NULL|       NULL|
|B00085T7O2|"""Catch 'em aliv...|",,A91QQ7IYBAK2Q,...| terrible forward...|I found the book ...|                NULL|                NULL|                NULL|                NULL|       NULL|
|B0007DMFZS|"""Beloved friend...|",

There are copies of the'review/text' null entries.

 Aside from the Title and the IDs, they have the same columns.  A qualitative examination reveals that they are about the same book even though the titles are different.

There are numerous other reviews of Lord of the Rings.  I believe that the NA values should be retained because they cannot be discarded.  The title can also be used as a similarity check.

In [27]:
from pyspark.sql.functions import col
filter_df = books_rating.filter(
    col("Title").contains("The Lord of the Rings")
).select("Title", "review/text").limit(100)

# Display result
filter_df.show(100, truncate=True)

+--------------------+--------------------+
|               Title|         review/text|
+--------------------+--------------------+
|The Lord of the R...|"I enjoy these ta...|
|The Lord of the R...|This drama comes ...|
|The Return of the...|Have the Rings fo...|
|The Return of the...|I highly recommen...|
|Gollum: How We Ma...|"Andy Serkis is t...|
|Gollum: How We Ma...|I guess I'm a bit...|
|The Lord of the R...|I've never attemp...|
|The Lord of the R...|This is one of th...|
|The Lord of the R...|This made an idea...|
|The Lord of the R...|This is one hefty...|
|The Lord of the R...|This book is a mu...|
|The Lord of the R...|This comprehensiv...|
|The Lord of the R...|I just got the bo...|
|The Lord of the R...|"Length:: 2:05 Mi...|
|The Lord of the R...|".""...has left t...|
|The Lord of the R...|I will not belabo...|
|The Lord of the R...|"It is amazing th...|
|The Lord of the R...|"Wayne G. Hammond...|
|The Lord of the R...|The content of th...|
|The Lord of the R...|"Two of th

Null values in subsample books.

In [28]:
# Check for null values in the 'review/text' column
Subsample_book = books_sample.filter(
    books_sample['review/text'].isNull() | (books_sample['review/text'].rlike('^\s*$'))
)
#Display sample books
Subsample_book.show(35)

+----------+--------------------+-----+-------+-----------+------------------+------------+-----------+--------------------+-----------+
|        Id|               Title|Price|User_id|profileName|review/helpfulness|review/score|review/time|      review/summary|review/text|
+----------+--------------------+-----+-------+-----------+------------------+------------+-----------+--------------------+-----------+
|B000I1VJLA|The Lord of the R...| NULL|   NULL|       NULL|               0/1|         5.0|  938563200|have only one wor...|       NULL|
+----------+--------------------+-----+-------+-----------+------------------+------------+-----------+--------------------+-----------+



#Jaccard Similarity


#Tokenisation
 To test the Jaccard Similarity technique, I will utilise a subsample of the dataset because it is too large.  After that, I'll attempt to top scale it to the entire dataset.  Additionally, I'll parallelise the Jaccard Similarity calculation using the MapReduce technique.

In [29]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

In [30]:
tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\W") # \W is a regex pattern that matches any non-word character


## Apply tokenization

### Subtask:
Apply the `RegexTokenizer` to the `review/text` column of your Spark DataFrame to get tokens.


**Meaning**
Apply the pre-defined RegexTokenizer to the books_sample DataFrame and display the schema of the resulting DataFrame to confirm the new column.



In [31]:
tokenized_df = tokenizer.transform(books_sample)
tokenized_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)




The error indicates that the `tokenizer` object was not defined. This is because the previous cell that defined it was not executed in the current runtime. I need to redefine the tokenizer before using it to transform the DataFrame.



In [32]:
from pyspark.ml.feature import RegexTokenizer

tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\W")
tokenized_df = tokenizer.transform(books_sample)
tokenized_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



**Reasoning**:
The error indicates that `books_sample` is not defined. This is because the cell where `books_sample` was created by sampling `books_rating` was not executed in the current runtime. I need to recreate the `books_sample` DataFrame before applying the tokenizer.



In [33]:
sample = 0.02
books_sample = books_rating.sample(fraction=sample, seed=42)

tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\W")
tokenized_df = tokenizer.transform(books_sample)
tokenized_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



**Reasoning**:
The error indicates that `books_rating` is not defined. This is because the cell where `books_rating` was loaded was not executed in the current runtime. I need to load the `books_rating` DataFrame before creating the sample and applying the tokenizer.



In [34]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer

spark = SparkSession.builder.appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()

books_rating = spark.read.csv("/content/data/Books_rating.csv", header=True, inferSchema=True)

sample = 0.02
books_sample = books_rating.sample(fraction=sample, seed=42)

tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\W")
tokenized_df = tokenizer.transform(books_sample)
tokenized_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



**Reasoning**:
The error indicates that the file path `/content/data/Books_rating.csv` does not exist. This is likely because the previous unzipping step was not executed in the current runtime, or the path is incorrect. I need to ensure the file is unzipped and then load the data. The previous unzipping code cell used `/content/data` as the extraction path, so the path in the loading code should be correct if the unzipping is done. I will include the unzipping step before loading the data.



In [36]:
import zipfile
import os
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer

zip_file_path = "/content/my_project/amazon-books-reviews.zip"
extract_path = "/content/data"

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

spark = SparkSession.builder.appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()

books_rating = spark.read.csv("/content/data/Books_rating.csv", header=True, inferSchema=True)

sample = 0.02
books_sample = books_rating.sample(fraction=sample, seed=42)

tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\W")
tokenized_df = tokenizer.transform(books_sample)
tokenized_df.printSchema()

Files extracted to: /content/data
root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



**Reasoning**:
The error indicates that the zip file `/content/amazon-books-reviews.zip` was not found. This means the Kaggle download step was not executed in the current runtime. I need to download the dataset using the Kaggle API before attempting to unzip and load it again.



In [37]:
import zipfile
import os
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer
import kaggle

# Download the dataset
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews -p /content/ --unzip

zip_file_path = "/content/amazon-books-reviews.zip"
extract_path = "/content/data"

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the file if it wasn't unzipped by kaggle command
if not os.path.exists(os.path.join(extract_path, 'Books_rating.csv')):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

spark = SparkSession.builder.appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()

books_rating = spark.read.csv(os.path.join(extract_path, "Books_rating.csv"), header=True, inferSchema=True)

sample = 0.02
books_sample = books_rating.sample(fraction=sample, seed=42)

tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\W")
tokenized_df = tokenizer.transform(books_sample)
tokenized_df.printSchema()

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 97% 1.03G/1.06G [00:03<00:00, 265MB/s]
100% 1.06G/1.06G [00:03<00:00, 292MB/s]
Files extracted to: /content/data
root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)




The error indicates that the zip file was not found even after the Kaggle download command. However, the Kaggle command includes the `--unzip` flag, which should extract the files directly to the specified path. Therefore, the manual unzipping step is likely redundant and causing the error because the zip file is no longer in the expected location after the Kaggle command extracts its contents. I should remove the manual unzipping and directly load the CSV from the directory specified in the Kaggle download command.



In [38]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer
import os
import kaggle

# Download the dataset
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews -p /content/data --unzip

spark = SparkSession.builder.appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()

books_rating = spark.read.csv("/content/data/Books_rating.csv", header=True, inferSchema=True)

sample = 0.02
books_sample = books_rating.sample(fraction=sample, seed=42)

tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\W")
tokenized_df = tokenizer.transform(books_sample)
tokenized_df.printSchema()

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 98% 1.04G/1.06G [00:16<00:01, 19.7MB/s]
100% 1.06G/1.06G [00:16<00:00, 68.0MB/s]
root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



## Remove stop words

### Subtask:
Use `StopWordsRemover` to remove common English stop words from the tokenized reviews.


**Reasoning**:
Use StopWordsRemover to remove common English stop words from the tokenized reviews.



In [39]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="reviews/tokens", outputCol="reviews/filtered_tokens")
filtered_df = remover.transform(tokenized_df)
filtered_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- reviews/tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- reviews/filtered_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



## Calculate jaccard similarity

### Subtask:
Implement a method to calculate the Jaccard similarity between the processed review tokens for pairs of books.



 Python function to calculate Jaccard similarity and register it as a Spark UDF. Then, join the DataFrame with itself to create pairs of books and apply the UDF to calculate the similarity between their filtered tokens.



In [40]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

def jaccard_similarity(list1, list2):
    if not list1 or not list2:
        return 0.0
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0.0

jaccard_udf = udf(jaccard_similarity, DoubleType())

# Create pairs of books for similarity calculation
book_pairs = filtered_df.alias("df1").join(
    filtered_df.alias("df2"),
    col("df1.Id") < col("df2.Id")
)

# Calculate Jaccard similarity for each pair
similarity_df = book_pairs.withColumn(
    "jaccard_similarity",
    jaccard_udf(col("df1.reviews/filtered_tokens"), col("df2.reviews/filtered_tokens"))
)

# Select relevant columns and show the results
similarity_df.select(
    col("df1.Id").alias("book1_id"),
    col("df2.Id").alias("book2_id"),
    col("jaccard_similarity")
).show(20)

+----------+----------+--------------------+
|  book1_id|  book2_id|  jaccard_similarity|
+----------+----------+--------------------+
|0595344550|B000NKGYMK|0.056338028169014086|
|0595344550|B000NKGYMK|0.013513513513513514|
|0595344550|157067051X|0.015151515151515152|
|0595344550|B0007DVHU2|0.009523809523809525|
|0595344550|0781810698|0.013888888888888888|
|0595344550|B0006D6DRK|  0.0425531914893617|
|0595344550|0934638160|0.013333333333333334|
|0595344550|0671551345|0.029850746268656716|
|0595344550|0671551345|0.011363636363636364|
|0595344550|1555714137| 0.05426356589147287|
|0595344550|B0000630MU|0.012195121951219513|
|0595344550|B0008852GG|0.017543859649122806|
|0595344550|B000MCKQRS|0.047619047619047616|
|0595344550|155558182X|  0.0380952380952381|
|0595344550|0882899228|0.020202020202020204|
|0595344550|B000O3QCH8| 0.05263157894736842|
|0595344550|B000O3QCH8|0.038834951456310676|
|0595344550|0836204271|0.023809523809523808|
|0595344550|0932813062|0.031914893617021274|
|059534455

In [41]:
 #from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
#from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

In [42]:
tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\W") # \W is a regex pattern that matches any non-word character

The text is tokenised into words.  Since this method uses sparks, I think it will be quicker than certain neural network-based methods, therefore I'm giving it a shot.  Although it will have certain drawbacks because it ignores compound terms, I believe the first strategy will benefit from it.

In [43]:
stopwords_remover = StopWordsRemover(inputCol="reviews/tokens", outputCol="tokens/nostopwords")

A predetermined list of English stop words is used by StopWordsRemover().  Common terms like "the," "is," "in," and others that don't have much meaning are eliminated.  This is significant because the number of words that two texts share determines the Jaccard Similarity.  The quantity of stop words in the texts, which are irrelevant to their meaning, will skew the Jaccard Similarity if they are not eliminated.

In [44]:
def remove_numbers(tokens):
    return [token for token in tokens if not token.isdigit()]

Frequently, the meaning of a review is not affected by numbers.  We can eliminate the numbers from the text because the score in our dataset is kept in a distinct column.

In [45]:
remove_numbers_udf = udf(remove_numbers, ArrayType(StringType())) # Specify the return type as ArrayType(StringType), meaning an array/list of strings


Because Sparks is not a Python object, it must be used in the pipeline as a user-defined function (UDF).

In [46]:
tokenized_books_sample = tokenizer.transform(books_sample) # Apply the tokenizer to the dataset
tokenized_reviews_nostopwords_sample = stopwords_remover.transform(tokenized_books_sample) # Remove the stop words from the tokenized dataset
tokenized_rw_nsw_nn_sample = tokenized_reviews_nostopwords_sample.withColumn("tokens/nostopwords/nonumbers", remove_numbers_udf(col("tokens/nostopwords"))) # Remove the numbers from the dataset



In [47]:

books_sample.show(10)
tokenized_rw_nsw_nn_sample.show(10, truncate=True)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|0595344550|Whispers of the W...|10.95| AUR0VA5H0C66C|"LoveToRead ""Act...|               1/2|         1.0| 1119225600|        Buyer beware|"This is a self-p...|
|B000NKGYMK|    Alaska Sourdough| NULL|A258YNWJW2264M|"Tessa F. Briggs ...|              8/14|         3.0| 1241827200|Not your quick re...|"After having a c...|
|B000NKGYMK|    Alaska Sourdough| NULL|A2WY5VMJQ0MM1A|                 Aoy|               0/0|         5.0| 1351641600|        Old and good|This book is wort...|
|157067051X|Voices from the 

To build a vocabulary of the tokens, we require CountVectorizer().  The most common tokens in the dataset will be compiled into a vocabulary.  The vocabulary's maximum size is determined by the vocabularySize.  The minimum quantity of documents containing a token in order for it to be incorporated into the vocabulary is known as the minDF parameter.  This is crucial since we wish to eliminate uncommon tokens that don't add to the texts' meaning.

In [48]:
from pyspark.ml.feature import CountVectorizer

df_safe = tokenized_rw_nsw_nn_sample \
    .filter(col("review/text").isNotNull())

cv = CountVectorizer(
    inputCol="tokens/nostopwords/nonumbers",
    outputCol="features",
    vocabSize=5000,
    minDF=10
)

model = cv.fit(df_safe)
vocab = model.vocabulary


In [49]:
top_words = model.vocabulary[:30]
print("Top 30 tokens (approx. by frequency):")
for i, token in enumerate(top_words, start=1):
    print(f"{i:2d}. {token}")

Top 30 tokens (approx. by frequency):
 1. book
 2. read
 3. one
 4. quot
 5. story
 6. like
 7. books
 8. time
 9. good
10. great
11. reading
12. life
13. first
14. well
15. many
16. much
17. people
18. love
19. really
20. also
21. characters
22. novel
23. even
24. get
25. way
26. author
27. world
28. think
29. written
30. years


The top 30 terms in the vocaboulary are roughly represented by this way.  However, since CountVectorizer() merely approximates the actual frequency of the top words, it does not display the frequency, I'm looking for

In [50]:
from pyspark.sql.functions import explode, col

exploded = df_safe.select(explode(col("tokens/nostopwords/nonumbers")).alias("token"))

# Filtering only the top-20 tokens
topK_set = set(model.vocabulary[:20])  # top list  20 from vocabulary
filtered = exploded.filter(col("token").isin(topK_set))

exact_counts = (filtered.groupBy("token").count().orderBy(col("count").desc()))
exact_counts.show(truncate=False)

+-------+-----+
|token  |count|
+-------+-----+
|book   |96856|
|read   |40403|
|one    |29119|
|quot   |26373|
|story  |20064|
|like   |18044|
|books  |14916|
|time   |14630|
|good   |14489|
|great  |14359|
|reading|13641|
|life   |13370|
|first  |12788|
|well   |12378|
|many   |12281|
|much   |11090|
|people |10652|
|love   |10610|
|really |10566|
|also   |10528|
+-------+-----+



In [51]:
threshold = 0.30 * size # % of the sample size and can be adjust
print(threshold)

18149.399999999998


In [52]:
highest_freq_tokens = (exact_counts.filter(col("count") > threshold).select("token")
      .rdd.flatMap(lambda x: x)
      .collect()
)


In [53]:
highest_freq_tokens

['book', 'read', 'one', 'quot', 'story']

#Scalability

We must filter out the entries that contain empty lists because Sparks does not function properly with them.  To accomplish this, we may use the size() and isNull() functions to determine whether the list is null or empty.

---



Filtering out stop words in the tokenised data.

In [64]:
def remove_custom_stopwords(tokens):
    return [t for t in tokens if t not in highest_freq_tokens]

remove_udf = udf(remove_custom_stopwords, ArrayType(StringType()))

tokenized_sample_2 = tokenized_rw_nsw_nn_sample.withColumn(
    "tokens_clean",
    remove_udf(col("tokens/nostopwords/nonumbers"))
)

In [68]:
from pyspark.sql.functions import size
df_safe_clean = tokenized_sample_2.filter(col("review/text").isNotNull())

empty_tokens_count = df_safe_clean.filter(
    (size(col("tokens_clean")) == 0) |
    (col("tokens_clean").isNull())
).count()

print(f"clean empty tokens: {empty_tokens_count}")

clean empty tokens: 106


Filtering rows with empty list becasue spark doesnot work well with lists.

In [69]:
cv = CountVectorizer(
    inputCol="tokens_clean",
    outputCol="features",
    vocabSize=5000,
    minDF=10
)

model2 = cv.fit(df_safe_clean)
vocab2 = model2.vocabulary


different top words

In [70]:
top20 = model2.vocabulary[:20]
print(top20)
for i, token in enumerate(top20, start=1):
    print(f"{i:2d}. {token}")

['like', 'books', 'time', 'good', 'great', 'reading', 'life', 'first', 'well', 'many', 'much', 'people', 'love', 'really', 'also', 'characters', 'novel', 'even', 'get', 'way']
 1. like
 2. books
 3. time
 4. good
 5. great
 6. reading
 7. life
 8. first
 9. well
10. many
11. much
12. people
13. love
14. really
15. also
16. characters
17. novel
18. even
19. get
20. way


In [72]:
# Removing Duplicates
df_safe = df_safe_clean.dropDuplicates(["review/text"])

#Local Sensitive Hashing
is a method that hashes similar input items into the same "buckets" with high probability. It is used to find similar items in large datasets efficiently.

In [73]:
sim_threshold = 0.6 # You can change it at your will


In [74]:
df_safe.show(10)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|      reviews/tokens|  tokens/nostopwords|tokens/nostopwords/nonumbers|        tokens_clean|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+
|0595229751|Tranceformers: Sh...| NULL|A3N4LVV9SE7BM8|"Shea Bower ""Dr....|             15/20|         5.0| 1159401600|           """Lights|        ..... Action|            [action]|            [action]|                    [action]|      

In [77]:

# Filter out rows where tokens_clean is empty
df_empty_Cleantoken = df_safe.filter((df_safe.tokens_clean.isNotNull()) & (size(df_safe.tokens_clean) > 0))

In [78]:
df_empty_Cleantoken = df_empty_Cleantoken.dropDuplicates(["tokens_clean"])

In [80]:
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(inputCol="tokens_clean", outputCol="features", numFeatures= 8192)
df_featurized_Cleantoken = hashingTF.transform(df_empty_Cleantoken)

In [81]:
df_featurized_Cleantoken.show(10)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|      reviews/tokens|  tokens/nostopwords|tokens/nostopwords/nonumbers|        tokens_clean|            features|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+--------------------+
|B0007HGBGI|       Black like me| NULL|          NULL|                NULL|              5/10|         5.0| 1031443200|Black Like Me Boo...|... ...09/07/02Co...|[09, 07, 02cor